# Day 22 Monkey Map

Urghr.. back to maps. I don't enjoy these so much. 

Looks like LOGO... rotate and move forward.
World wraps around - fall off a tile, reappear other side
Can't move through walls - just butt up against them

Answer needs row, column, facing
Rows start from 1 at the top and count downward; 
Columns start from 1 at the left and count rightward. 
Facing is 0 for right (>), 1 for down (v), 2 for left (<), and 3 for up (^). 

The final password is the sum of 1000 times the row, 4 times the column, and the facing.


In [1]:
testData = """        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5"""

import re
from pprint import pprint


VOID = ' '
TILE = '.'
WALL = '#'

RIGHT = 0
DOWN = 1
LEFT = 2
UP = 3

class Maze:
    def __init__(self,input:str):
        self.maze = [] #let's use Row Column referencing
        self.height = 0
        self.width = 0
        self.instructions = []
        self.position = (0,0)
        self.direction = RIGHT

        mazeParseComplete = False
        for l in input.splitlines():
            if l == "":
                mazeParseComplete = True
            elif mazeParseComplete:
                self.instructions = re.split('(\d+)',l) #weirdly contains empty string at front and back... deal with this later
            else:
                row = []
                if self.width > 0:
                    l = l.ljust(self.width, ' ') #janky input file isn't fully padded
                else:
                    self.width = len(l)
                self.maze.append(list(l))
        self.height = len(self.maze)
        #self.width = len(self.maze[0])
        print('Maze initialised with H='+str(self.height)+' W='+str(self.width))

    
    def setToStartPosition(self):
        #You begin the path in the leftmost open tile of the top row of tiles. 
        #Initially, you are facing to the right 
        for c in range(self.width):
            cell = self.maze[0][c]
            if cell == TILE:
                self.position = (0,c)
                break
        else:
            raise Exception('Tile not found in first row')


    def runTheMaze(self)->int:
        self.setToStartPosition()
        for instruction in self.instructions:
            #Need to determine if length or turn direction, and call appropriate function
            #print('Instruction: ' + instruction)
            if instruction == '':
                pass #skip
            elif instruction == 'L' or instruction == 'R':
                self.turn(instruction)
            else:
                self.walk(int(instruction))
                #print(self)
        #The final password is the sum of 1000 times the row, 4 times the column, and the facing.
        print('Final position:'+str(self.position)+' facing direction:'+str(self.direction))
        return 1000*(self.position[0]+1) + 4*(self.position[1]+1) + self.direction


    def walk(self,length:int):
        #walk step wise, until out of length, or hit a wall. World-wrap on the tiles.
        if self.direction == LEFT:
            while length > 0:
                nextCellPos = (self.position[0],self.position[1]-1)
                #detect wrap
                if self.position[1]-1 < 0 or self.maze[nextCellPos[0]][nextCellPos[1]]==VOID:
                    #find the rightmost tile in this row
                    x = self.width - 1
                    while x > 0:
                        nextCellPos = (self.position[0],x)
                        nextCell = self.maze[nextCellPos[0]][nextCellPos[1]]
                        if nextCell != VOID:
                            break
                        x -= 1
                else:
                    nextCell = self.maze[nextCellPos[0]][nextCellPos[1]]
                
                if nextCell == TILE:
                    self.markPath()
                    self.position = nextCellPos
                elif nextCell == WALL:
                    #stop
                    length = 0
                length -= 1


        elif self.direction == RIGHT:
            while length > 0:
                nextCellPos = (self.position[0],self.position[1]+1)
                #detect wrap
                if self.position[1]+1 >= self.width or self.maze[nextCellPos[0]][nextCellPos[1]]==VOID:
                    #find the rightmost tile in this row
                    x = 0
                    while x < self.width:
                        nextCellPos = (self.position[0],x)
                        nextCell = self.maze[nextCellPos[0]][nextCellPos[1]]
                        if nextCell != VOID:
                            break
                        x += 1
                else:
                    nextCell = self.maze[nextCellPos[0]][nextCellPos[1]]
                
                if nextCell == TILE:
                    self.markPath()
                    self.position = nextCellPos
                elif nextCell == WALL:
                    #stop
                    length = 0
                length -= 1


        elif self.direction == UP:
            while length > 0:
                nextCellPos = (self.position[0]-1,self.position[1])
                #detect wrap
                if self.position[0]-1 < 0 or self.maze[nextCellPos[0]][nextCellPos[1]]==VOID:
                    #find the rightmost tile in this row
                    x = self.height - 1
                    while x > 0:
                        nextCellPos = (x,self.position[1])
                        nextCell = self.maze[nextCellPos[0]][nextCellPos[1]]
                        if nextCell != VOID:
                            break
                        x -= 1
                else:
                    nextCell = self.maze[nextCellPos[0]][nextCellPos[1]]
                
                if nextCell == TILE:
                    self.markPath()
                    self.position = nextCellPos
                elif nextCell == WALL:
                    #stop
                    length = 0
                length -= 1

        elif self.direction == DOWN:

            while length > 0:
                nextCellPos = (self.position[0]+1,self.position[1])
                #detect wrap
                if self.position[0]+1 >= self.height or self.maze[nextCellPos[0]][nextCellPos[1]]==VOID:
                    #find the rightmost tile in this row
                    x = 0
                    while x < self.height:
                        nextCellPos = (x,self.position[1]) 
                        nextCell = self.maze[nextCellPos[0]][nextCellPos[1]]
                        if nextCell != VOID:
                            break
                        x += 1
                else:
                    nextCell = self.maze[nextCellPos[0]][nextCellPos[1]]
                
                if nextCell == TILE:
                    self.markPath()
                    self.position = nextCellPos
                elif nextCell == WALL:
                    #stop
                    length = 0
                length -= 1

    def turn(self,direction:str):
        if direction=='L':
            self.direction = (self.direction - 1) % 4
        elif direction=='R':
            self.direction = (self.direction + 1) % 4
        else:
            raise Exception('Unknown direction: '+direction)

    def markPath(self):
        #self.maze[self.position[0]][self.position[1]] = self.direction
        pass
    
    def __str__(self):
        output = ""
        for r in self.maze:
            for c in r:
                if isinstance(c, int):
                    output += str(c)
                else:
                    output += c
            output +=  '\n'
        return output

#tests
tm = Maze(testData)
print(tm.instructions)
#print(tm)
print(tm.runTheMaze())




Maze initialised with H=12 W=12
['', '10', 'R', '5', 'L', '5', 'R', '10', 'L', '4', 'R', '5', 'L', '5', '']
Final position:(5, 7) facing direction:0
6032


In [2]:
puzzleinput = open('day22input.txt').read()
m = Maze(puzzleinput)
print(m.runTheMaze())

Maze initialised with H=200 W=150
Final position:(148, 33) facing direction:2
149138
